Equity Index Yield Strategy

Inputs:
1. Ticker
2. Expiration Date
3. Desired Yield Spread to Treasuries
4. Desired OTM Percentage of first sold put ('break_point')
5. Bond Asset and YTM
6. Oakhurst Annual Management Fee

Outputs:
1. Payoff Diagram
2. Trade Details (Strikes, Premiums, Buy/Sell, Cap, Cushion)
3. Expected Interest
4. Returns DataFrame (can be customized to show prices everty X dollars)

***RETURNS ARE CALCULATED NET OF FEES***

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### Define function to organize options data ###

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 0)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days# / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])
    options['PUT']=np.where(options['CALL']==True,'False','True')
    
    print(exps)
    
    return options

In [ ]:
############################

### Source and oganize data, print available options expiration dates ### 

ticker='SPY'

data=yf.download(ticker,period='1y')
last=np.round(data['Adj Close'][-1],3)
chain=options_chain(ticker)

############################

In [ ]:
############################

### Set trade parameters ###

expiry='2022-03-18' #DESIRED EXPIRATION DATE

spread=0.04 #DESIRED SPREAD TO T-BILL RATE

break_point=0.15 #DESIRED OTM PERCDENTAGE OF FIRST SOLD PUT

up_down=0.4 #Define range of prices for options payoff diagram and returns table

# Set Bond Asset (Treasury Bill, Muni, Bullet Shares Bond ETF)
bond='T-Bills'

rate=0.0010 #1-yr T-Bill Rate; currently hardcoded. -->Action: set code to fetch automatically
#Source: Charles Schwab BondSource

# Set Management Fee
fee=0.0040

############################

In [ ]:
### Main function to caluclate strike, trade details and create visulations ### 

#Split raw options data into Calls / Puts and filtered for only desired expiration
calls=chain.loc[chain.CALL==True]
puts=chain.loc[chain.CALL!=True]

active_calls=calls.loc[calls['expirationDate']==expiry]
active_puts=puts.loc[puts['expirationDate']==expiry]

#Determine ATM Put Strike & ATM Put Strike Premium
atm=active_puts['strike'].iloc[np.argmin(np.abs((active_puts['strike']-last)))]

high_strike=active_puts['strike'].iloc[np.argmin(np.abs((active_puts['strike']-(atm*(1-break_point)))))]
high_put_prem=active_puts['mark'].loc[active_puts['strike']==high_strike].iloc[0]

atm,high_strike,high_put_prem

#Calculate required interest based upon spread to T-Bill Rate
notional=atm*100 #notional value of ATM options
total_int=notional*spread

notional,total_int

#Calculate Long OTM Put Strike & Long OTM Put Premium
target_otm_put_strike=np.round(high_strike-(total_int/100),0)
long_otm_put_strike=active_puts['strike'].iloc[np.argmin(np.abs(target_otm_put_strike-active_puts['strike']))]
long_otm_put_prem=active_puts['mark'].loc[active_puts['strike']==long_otm_put_strike].iloc[0]

long_otm_put_strike,long_otm_put_prem

#Calculate Short OTM Put Strike & Long OTM Put Premium
prem_require=(total_int/100)-(high_put_prem-long_otm_put_prem)
short_otm_put_strike=active_puts['strike'].iloc[np.argmin(np.abs(prem_require-active_puts['mark']))]
short_otm_put_prem=active_puts['mark'].loc[active_puts['strike']==short_otm_put_strike].iloc[0]

short_otm_put_strike,short_otm_put_prem

#Calcualte net cost of options trade
net=(high_put_prem*-1)+(long_otm_put_prem)+(short_otm_put_prem*-1)
net_cost=net*100
cred_deb=np.where(net_cost>0,'Debit','Credit')

#Caluclate Calendar Days Until Expirations and Expected Interest Received from Bond Asset
from datetime import datetime
today=datetime.now()
today_str=today.strftime('%Y-%m-%d')
biz_days=np.busday_count(today.strftime('%Y-%m-%d'),expiry)
exp=datetime.strptime(expiry,'%Y-%m-%d')
cal_days=(exp-today).days
interest=(cal_days/365)*rate*notional

#Calculate Oakhurst Management Fee
mgt_fee_pct=(cal_days/365)*fee #Management Fee expressed as percentage over life of options
mgt_fee=mgt_fee_pct*notional #Management Fee in $$ over life of options

#Calculate Buffer and Strategy Yield
buffer=((atm-short_otm_put_strike)/atm)-mgt_fee_pct
yld_protect=((atm-high_strike)/atm)-mgt_fee_pct
strat_yield=((np.abs(net_cost)+interest)/notional)-mgt_fee_pct

net_cost,buffer,yld_protect,strat_yield,interest,mgt_fee

# Function to calculate options payoffs at EXPIRY
def call_payoff(stock_range,strike,premium):
    return np.where(stock_range>strike,stock_range-strike,0)-premium
def put_payoff(stock_range,strike,premium):
    return np.where(stock_range<strike,strike-stock_range,0)-premium

# Define stock price range at expiration
up_down=0.4
stock_range=np.arange((1-up_down)*last,(1+up_down)*last,1)

# Calculate payoffs for individual legs
payoff_high_put=put_payoff(stock_range,high_strike,high_put_prem)*-1
payoff_long_put=put_payoff(stock_range,long_otm_put_strike,long_otm_put_prem)*1
payoff_short_put=put_payoff(stock_range,short_otm_put_strike,short_otm_put_prem)*-1

# Calculate Strategy Payoff
strategy=((payoff_high_put+payoff_long_put+payoff_short_put+(interest/100))/last)-mgt_fee_pct
buy_hold_ret=(stock_range/last)-1

# Create DataFrame of Stock Prices every $25
range_by25=np.arange(stock_range[0],stock_range[len(stock_range)-1],25)
def myround(x, base=5):
    return base * np.around(x/base)
stock_range_25=myround(range_by25,base=25).astype(int)

# Calculate strategy payoff with alternative stock range
sp_high=put_payoff(stock_range_25,high_strike,high_put_prem)*-1
lp=put_payoff(stock_range_25,long_otm_put_strike,long_otm_put_prem)*1
sp_low=put_payoff(stock_range_25,short_otm_put_strike,short_otm_put_prem)*-1

strat=((sp_high+lp+sp_low+(interest/100))/last)-mgt_fee_pct
bh_ret=(stock_range_25/last)-1

data=pd.DataFrame({'{} Price'.format(ticker):stock_range_25,'Strategy Return':strat,
               '{} Return'.format(ticker):bh_ret}).sort_values(by='{} Price'.format(ticker),ascending=False)
data.set_index('{} Price'.format(ticker),inplace=True)
styled_data=data.style.format({'Strategy Return': "{:.2%}",'{} Return'.format(ticker): "{:.2%}"})

#Save DataFrame of Returns for Strategy vs Buy-Hold every X dollars
styled_data.to_excel('Oakhurst Equity Index Yield Strategy with Buffer for {}.xlsx'.format(ticker))

#Create Visualization
# import mplcyberpunk
plt.style.use('fivethirtyeight')

# plt.style.use('fivethirtyeight')
plt.figure(figsize=(14,8))

from matplotlib.ticker import PercentFormatter
plt.yticks(np.arange(-up_down,up_down+0.01,.1),fontsize=16)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xticks(fontsize=16)
plt.gca().xaxis.set_major_formatter('${x:1.0f}')

plt.plot(stock_range,strategy,c='green',lw=4,label='Strategy Return (net of fees at {:.2%}/yr.)'.format(fee))
plt.plot(stock_range,buy_hold_ret,c='gray',ls='dotted',lw=4,label='{} Price Return'.format(ticker))
plt.scatter(last,0,s=100,marker='D',c='black',label='Current {} price: ${}'.format(ticker,last))

plt.hlines(y=0,xmin=stock_range[0],xmax=stock_range[len(stock_range)-1],color='red',lw=2)

plt.legend(loc='upper left',fontsize=14)
plt.ylabel('Return',fontsize=18)
plt.xlabel('{} Price'.format(ticker),fontsize=18)
plt.annotate('{:.2%} Downside Buffer'.format(buffer),xy=(short_otm_put_strike,0),xytext=(-200,25),
        textcoords='offset points',arrowprops=dict(facecolor='navy',shrink=0.05),
        weight='bold',size=12,color='navy')
plt.annotate('{:.2%} Income Buffer'.format(yld_protect),xy=(high_strike,strat_yield),xytext=(-100,50),
        textcoords='offset points',arrowprops=dict(facecolor='navy',shrink=0.05),
        weight='bold',size=12,color='navy')
plt.annotate('Startegy Yield: {:.2%} (net)'.format(strat_yield),xy=(atm*1.25,strat_yield),xytext=(-50,50),
        textcoords='offset points',arrowprops=dict(facecolor='navy',shrink=0.05),
        weight='bold',size=12,color='navy')

plt.suptitle('Oakhurst Equity Index Yield Strategy with Buffer for {}'.format(ticker),fontsize=18)
plt.title('Expiration: {}, ({:.1f} months)'.format(expiry,cal_days/30),fontsize=16)
plt.savefig('Oakhurst Equity Index Yield Strategy with Buffer for {} at {} expiry.png'.format(ticker,expiry))
plt.show();

print('Trade Details:')
print('-Sell 1 {} {:.0f} Put at ${:.2f}'.format(ticker,high_strike,high_put_prem))
print('-Buy 1 {} {:.0f} Put at ${:.2f}'.format(ticker, long_otm_put_strike,long_otm_put_prem))
print('-Sell 1 {} {:.0f} Put at ${:.2f}'.format(ticker, short_otm_put_strike,short_otm_put_prem))
print('-Buy ${:.2f} of {} due closest to {}'.format(notional,bond,expiry))
trade_cost=((long_otm_put_prem-high_put_prem-short_otm_put_prem)*100*-1)
print('Options Income: ${:.2f}'.format(trade_cost))
print('Expected Bond Interest: ${:.2f}'.format(interest))
print('Total Income: ${:.2f} ({:.2%}) (*Net of fees)'.format(trade_cost+interest-mgt_fee,strat_yield))
print('Management Fee (through expiry): ${:.2f}'.format(mgt_fee))
print('Income Protection Buffer: {:.1%}'.format(yld_protect))
print('Total Buffer: {:.1%}'.format(buffer))

styled_data